In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import random
import joblib
import seaborn as sns
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from scipy.signal import welch, resample, medfilt
from scipy.interpolate import splev, splrep, interp1d
print(np.__version__)
sns.set_theme()

1.21.3


In [2]:
def load_data(file_name, base_dir = './'):
    with open(os.path.join(base_dir, file_name), 'rb') as f: # read preprocessing result
        ecg_data = joblib.load(f)
    X, y, groups = ecg_data["X"], ecg_data["y"], ecg_data["groups"]
    X = np.array(X)
    y = np.array(y)
    del(ecg_data)
    return X, y, groups

def load_data(base_dir = './paper/dataset', file_name = "apnea-ecg-physio-edr-train.pkl"):
    ir = 3 # interpolate interval
    before = 2
    after = 2
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, file_name), 'rb') as f: # read preprocessing result
        apnea_ecg_train = joblib.load(f)
    
    print('\n Processing training data...')
    t = time.time()
    x_train = []
    o_train, edr_train, y_train = apnea_ecg_train["o_train"], apnea_ecg_train["edr_train"], apnea_ecg_train["y_train"]
    groups_train = apnea_ecg_train["groups_train"]
    for i in range(len(o_train)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_train[i]
		# Curve interpolation
        if edr_train[i][0] == []:
            print("Empty EDR signal!")
            continue
        rri_interp_signal = splev(tm, splrep(rri_tm, rri_signal, k=3), ext=0) 
        ampl_interp_signal = splev(tm, splrep(ampl_tm, ampl_siganl, k=3), ext=0)
        edr_interp_signal = resample(edr_train[i][0], 900)
        x_train.append([rri_interp_signal, ampl_interp_signal, edr_interp_signal])
    x_train = np.array(x_train, dtype="float32").transpose((0, 2, 1)) # convert to numpy format
    y_train = np.array(y_train, dtype="float32")
    print('\n Done in ' + str(time.time() - t) + 's!')
    del(apnea_ecg_train)

    with open(os.path.join(base_dir, "apnea-ecg-physio-edr-test.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg_test = joblib.load(f) 

    print('\n Processing testing data...')
    t = time.time()
    x_test = []
    o_test, edr_test, y_test = apnea_ecg_test["o_test"], apnea_ecg_test["edr_test"], apnea_ecg_test["y_test"]
    groups_test = apnea_ecg_test["groups_test"]
    for i in range(len(o_test)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_test[i]
		# Curve interpolation
        if edr_test[i][0] == []:
            print("Empty EDR signal!")
            continue
        rri_interp_signal = splev(tm, splrep(rri_tm, rri_signal, k=3), ext=0)
        ampl_interp_signal = splev(tm, splrep(ampl_tm, ampl_siganl, k=3), ext=0)
        edr_interp_signal = resample(edr_test[i][0], 900)
        x_test.append([rri_interp_signal, ampl_interp_signal, edr_interp_signal])
    x_test = np.array(x_test, dtype="float32").transpose((0, 2, 1))
    y_test = np.array(y_test, dtype="float32")
    print('\n Done in ' + str(time.time() - t) + 's!')
    del(apnea_ecg_test)

    return x_train, y_train, groups_train, x_test, y_test, groups_test

def load_data_cic(file_name, base_dir = './'):
    with open(os.path.join(base_dir, file_name), 'rb') as f: # read preprocessing result
        data = joblib.load(f)
    X, sleep_stages, sleep_apnea, groups = data["X"], data["sleep_stages"], data['sleep_apnea'], data["groups"]
    X = np.array(X)
    sleep_stages = np.array(sleep_stages)
    sleep_apnea = np.array(sleep_apnea)
    del(data)
    return X, sleep_stages, sleep_apnea, groups

def load_data_cic_new(file_name, base_dir = './'):
    with open(os.path.join(base_dir, file_name), 'rb') as f: # read preprocessing result
        data = joblib.load(f)
    X, sleep_stages, sleep_apnea, sleep_apnea_locs, groups = data["X"], data["sleep_stages"], data['sleep_apnea'], data["sleep_apnea_locs"], data["groups"]
    X = np.array(X)
    sleep_stages = np.array(sleep_stages)
    sleep_apnea = np.array(sleep_apnea)
    del(data)
    return X, sleep_stages, sleep_apnea, sleep_apnea_locs, groups

def create_CNN(input_shape, normalize=False, n_class=6, 
                    CNN_n_layers=2, CNN_kernel_size=[5,5], CNN_n_nodes=[32,64], pooling_size=[3,3], cnn_strides=[2,2],
                    dil_CNN_n_layers=2, dil_CNN_kernel_size=[5,5], dil_CNN_n_nodes=[32,64], dil_pooling_size=[3,3], dil_cnn_strides=[1,1], 
                    CNN_n_feature=32, drop_rate=0.2, weight=1e-3):

    inputs = layers.Input(shape=input_shape)
    if normalize == True:
        x = layers.BatchNormalization()(inputs)
    if normalize == False:
        x = inputs

	# Conv
    for i in range(CNN_n_layers):
        conv_layer = layers.Conv2D(CNN_n_nodes[i], kernel_size=(CNN_kernel_size[i],1), strides=(cnn_strides[i],1), padding="same", activation="relu", kernel_initializer="he_normal",
               kernel_regularizer=tf.keras.regularizers.l2(weight), bias_regularizer=tf.keras.regularizers.l2(weight), name=str('normal_conv_'+str(i)))
        x = conv_layer(x)
        max_pooling = layers.MaxPooling2D(pool_size=(pooling_size[i],1))
        x = max_pooling(x)


    #Dil conv
    for i in range(dil_CNN_n_layers):
        shape = x.shape.as_list()
        print(shape)
        dil_rate = int((shape[1]/2 - 3)/ 2) + 2
        print('dilation rate = ', dil_rate)
        conv_layer = layers.Conv2D(dil_CNN_n_nodes[i], kernel_size=(dil_CNN_kernel_size[i],1), strides=(dil_cnn_strides[i],1), dilation_rate=(dil_rate, 1), 
                padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=tf.keras.regularizers.l2(weight), bias_regularizer=tf.keras.regularizers.l2(weight), name=str('dilation_conv_'+str(i)))
        x = conv_layer(x)
        x = layers.Dropout(drop_rate)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(CNN_n_feature*4, activation="relu")(x)
    x = layers.Dense(CNN_n_feature, activation="relu")(x)
    outputs = layers.Dense(n_class, activation="softmax")(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    return(model)

def convert_class(y, n_class_ori=6, n_class=6):
    if n_class_ori==6:
        if n_class==6:
            return(y)
        y_conv = np.copy(y)
        if n_class==5: # merge n3 and n4
            y_conv[np.where(y==4)[0]] = 3
            y_conv[np.where(y==5)[0]] = 4
            return(y_conv)
        if n_class==4: #merge n3 and n4, n1 and n2
            y_conv[np.where(y==2)[0]] = 1
            y_conv[np.where(y==3)[0]] = 2
            y_conv[np.where(y==4)[0]] = 2
            y_conv[np.where(y==5)[0]] = 3
            return(y_conv)
        if n_class==3:
            y_conv[np.where(y==2)[0]] = 1
            y_conv[np.where(y==3)[0]] = 1
            y_conv[np.where(y==4)[0]] = 1
            y_conv[np.where(y==5)[0]] = 2
            return(y_conv)
    if n_class_ori==5:
        if n_class==5:
            return(y)
        y_conv = np.copy(y)
        if n_class==4: #merge n3 and n4
            y_conv[np.where(y==3)[0]] = 2
            y_conv[np.where(y==4)[0]] = 3
            return(y_conv)
        if n_class==3: #merge n1 and n2, n3 and n4
            y_conv[np.where(y==2)[0]] = 1
            y_conv[np.where(y==3)[0]] = 2
            y_conv[np.where(y==4)[0]] = 2
            return(y_conv)


def balance_class(X, y):
    new_X = np.copy(X)
    new_y = np.copy(y)
    n_class = np.max(y) + 1
    repartition = list(np.sum(np.array(to_categorical(y)), axis=0))
    less_rpz_class = repartition.index(min(repartition))
    for i in np.arange(n_class):
        if i == less_rpz_class:
            continue
        too_much = repartition[i] - repartition[less_rpz_class] 
        idx = np.where(new_y==i)[0]
        idx_to_remove = random.sample(range(0, len(idx)), int(too_much))
        new_X = np.delete(new_X, idx[idx_to_remove], axis = 0)
        new_y = np.delete(new_y, idx[idx_to_remove], axis = 0)
    return(new_X, new_y)

def convert_to_LSTM(X, y, groups, n_class):
    y_conv = np.copy(y)
    if n_class==5: # merge n3 and n4
            y_conv[np.where(y==4)[0]] = 3
            y_conv[np.where(y==5)[0]] = 4
            y = y_conv
    if n_class==4: #merge n3 and n4, n1 and n2
            y_conv[np.where(y==2)[0]] = 1
            y_conv[np.where(y==3)[0]] = 2
            y_conv[np.where(y==4)[0]] = 2
            y_conv[np.where(y==5)[0]] = 3
            y = y_conv

    y = np.array(to_categorical(y))

    patient_id = [groups[0][0]]
    for i in range(len(groups)):
            if groups[i][0] == patient_id[-1]:
                    continue
            patient_id.append(groups[i][0])

    ###########################
    # Remove patient 4 because bad
    ###########################

    X_lstm = []
    y_lstm = []
    for j in range(len(patient_id)):
            X_lstm_loc = []
            y_lstm_loc = []
            for i in range(X.shape[0]):
                    if groups[i][0] == patient_id[j]:
                            if X_lstm_loc == []:
                                    X_lstm_loc = np.expand_dims(X[i], 0)
                                    y_lstm_loc = np.expand_dims(y[i], 0)
                            else:
                                    X_lstm_loc = np.concatenate((X_lstm_loc, np.expand_dims(X[i], 0)))
                                    y_lstm_loc = np.concatenate((y_lstm_loc, np.expand_dims(y[i], 0)))
            X_lstm.append(np.expand_dims(X_lstm_loc, -1))
            y_lstm.append(y_lstm_loc)

    max_length = 0
    for i in range(len(X_lstm)):
        if X_lstm[i].shape[0] > max_length:
            max_length = X_lstm[i].shape[0]
    
    for i in range(len(X_lstm)):
        padding = int(max_length - X_lstm[i].shape[0])
        conc = np.concatenate((np.zeros((max_length - padding, 1)), np.ones((padding, 1))))
        X_lstm[i] = np.concatenate((X_lstm[i], np.zeros((padding, X_lstm[i].shape[1], 3, 1))))
        y_lstm[i] = np.concatenate((y_lstm[i], np.zeros((padding, n_class))))
        y_lstm[i] = np.concatenate((y_lstm[i], conc), axis = -1)
    return(np.array(X_lstm), np.array(y_lstm))
            

def conf_matrix_CNN(X_test, y_test, model):
        Y_test = y_test
        y_pred = model.predict(X_test)
        Y_pred = y_pred
        y_test_matrix = np.zeros(len(Y_test))
        y_pred_matrix = np.zeros(len(Y_pred))
        for i in range(len(y_test_matrix)):
                y_test_matrix[i] = np.argmax(Y_test[i])
                y_pred_matrix[i] = np.argmax(Y_pred[i])
        matrix = confusion_matrix(y_test_matrix, y_pred_matrix, normalize='pred')
        sns.heatmap(matrix, annot=True, fmt='.2%', cmap='Blues')
        return(matrix)

def conf_matrix_LSTM(X_test, y_test, model):
        Y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1], y_test.shape[2])
        y_pred = model.predict(X_test)
        Y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
        y_test_matrix = np.zeros(len(Y_test))
        y_pred_matrix = np.zeros(len(Y_pred))
        for i in range(len(y_test_matrix)):
                y_test_matrix[i] = np.argmax(Y_test[i])
                y_pred_matrix[i] = np.argmax(Y_pred[i])
        matrix = confusion_matrix(y_test_matrix, y_pred_matrix, normalize='pred')
        sns.heatmap(matrix, annot=True, fmt='.2%', cmap='Blues')
        plt.show()
        return(matrix)

def convert_to_CNN(y, n_class_ori, n_class):
    y_cnn = convert_class(y, n_class_ori, n_class)
    y_cnn = np.array(to_categorical(y_cnn))
    return(y_cnn)

def get_psd(x):
    fs, psd = welch(x, fs=3)
    return(fs,psd)

def accu_sensi_speci(y_pred, y_truth):
    y_p = np.zeros(len(y_pred))
    y_t = np.zeros(len(y_pred))
    for i in range(len(y_p)):
        if y_pred[i,0] > y_pred[i,1]:
            y_p[i] = 0
        else:
            y_p[i] = 1
    for i in range(len(y_t)):
        if y_truth[i,0] > y_truth[i,1]:
            y_t[i] = 0
        else:
            y_t[i] = 1
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len(y_t)):
        if y_t[i]==y_p[i]==1:
            tp+=1
        if y_t[i]==1 and y_p[i]==0:
            fn+=1
        if y_t[i]==0 and y_p[i]==1:
            fp+=1
        if y_t[i]==y_p[i]==0:
            tn+=1
    sensi = tp / (tp + fn)
    speci = tn / (tn + fp)
    accu = np.sum(y_t==y_p) / len(y_p)
    return(accu, sensi, speci)

def harmonize_data(X,y):
    _, class_0_idx = np.where(y==0)
    _, class_1_idx = np.where(y==1)

    class_0_idx = np.where(class_0_idx == 1)
    class_1_idx = np.where(class_1_idx == 1)

    idx_0 = random.choices(class_0_idx[0], k=int(len(class_1_idx[0])*1.5))
    idx = np.concatenate([idx_0, class_1_idx[0]])
    X = X[idx]
    y = y[idx]
    
    return(X,y)


class GradCAM:
    def __init__(self, model, classIdx, layerName):
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName

    def compute_heatmap(self, image, eps=1e-8):
        gradModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output, self.model.output])
		
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]
        grads = tape.gradient(loss, convOutputs)
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]
        weights = tf.reduce_mean(guidedGrads, axis=(0,1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")
        return(heatmap)

    def correlation(self, image, truth):
        pred = self.model.predict(image)
        i = np.argmax(pred[0])
        gradcam = GradCAM(self.model, i, "Conv3")
        heatmap = gradcam.compute_heatmap(image)
        title = "class = " + str(truth) + ', pred = ' + str(np.round(np.squeeze(pred), 2))
        x = np.arange(900)
        rri_weights = (heatmap[:,0] - np.mean(heatmap[:,0])) / (np.std(heatmap[:,0]) * len(heatmap[:,0]))
        rra_weights = (heatmap[:,1] - np.mean(heatmap[:,1])) / (np.std(heatmap[:,1]) * len(heatmap[:,1]))
        edr_weights = (heatmap[:,2] - np.mean(heatmap[:,2])) / (np.std(heatmap[:,2]) * len(heatmap[:,2]))

        rri_norm = (image[0,:,0,0] - np.mean(image[0,:,0,0])) / (np.std(image[0,:,0,0]) * len(image[0,:,0,0]))
        rra_norm = (image[0,:,1,0] - np.mean(image[0,:,1,0])) / (np.std(image[0,:,1,0]) * len(image[0,:,1,0]))
        edr_norm = (image[0,:,2,0] - np.mean(image[0,:,2,0])) / (np.std(image[0,:,2,0]) * len(image[0,:,2,0]))

        rri_correl = np.correlate(rri_norm, rri_weights)
        rra_correl = np.correlate(rra_norm, rra_weights)
        edr_correl = np.correlate(edr_norm, edr_weights)

        correl = np.zeros((3,3))
        signals = [rri_weights, rra_weights, edr_weights]
        for i in range(3):
            for j in range(3):
                correl[i,j] = np.correlate(signals[i], signals[j])
        correl = np.multiply(correl, 1/correl[0,0])
        print(correl)
        return()


    def plot_heatmap(self, image, truth):
        pred = self.model.predict(image)
        i = np.argmax(pred[0])
        gradcam = GradCAM(self.model, i, "Conv3")
        heatmap = gradcam.compute_heatmap(image)
        title = "class = " + str(truth) + ', pred = ' + str(np.round(np.squeeze(pred), 2))
        x = np.arange(900)
        rri_weights = (heatmap[:,0] - np.min(heatmap[:,0]))
        rri_weights = rri_weights / np.max(rri_weights) * (np.max(image[:,:,0]) - np.min(image[:,:,0])) + np.max(image[:,:,0])
        rra_weights = (heatmap[:,1] - np.min(heatmap[:,1]))
        rra_weights = rra_weights / np.max(rra_weights) * (np.max(image[:,:,1]) - np.min(image[:,:,1])) + np.max(image[:,:,1])
        plt.subplot(211)
        plt.plot(x, rri_weights, x, np.squeeze(image[:,:,0]))
        plt.legend(['Weights', 'RRi'])
        plt.xlabel('Sample')
        plt.xlim([20,880])
        #plt.vlines(x=[360,540], ymin=np.min(image[:,:,0]), ymax=np.max(rri_weights), color='red')
        plt.subplot(212)
        plt.plot(x, rra_weights, x, np.squeeze(image[:,:,1]))
        plt.legend(['Weights', 'RRamp'])
        plt.xlabel('Sample')
        plt.xlim([20,880])
        #plt.vlines(x=[360,540], ymin=np.min(image[::,:,0]), ymax=np.max(rra_weights), color='red')
        if image.shape[2] == 3:
            edr_weights = (heatmap[:,2] - np.min(heatmap[:,2]))
            edr_weights = edr_weights / np.max(edr_weights) * (np.max(image[:,:,2]) - np.min(image[:,:,2])) + np.max(image[:,:,2])
            plt.subplot(311)
            plt.plot(x, rri_weights, x, np.squeeze(image[:,:,0]))
            plt.legend(['Weights', 'RRi'])
            plt.xlabel('Sample')
            plt.xlim([20,880])
            #plt.vlines(x=[360,540], ymin=np.min(image[:,:,0]), ymax=np.max(rri_weights), color='red')
            plt.subplot(312)
            plt.plot(x, rra_weights, x, np.squeeze(image[:,:,1]))
            plt.legend(['Weights', 'RRamp'])
            plt.xlabel('Sample')
            plt.xlim([20,880])
            plt.suptitle(title)
            plt.subplot(313)
            plt.plot(x, edr_weights, x, np.squeeze(image[:,:,2]))
            plt.legend(['Weights', 'EDR'])
            plt.xlabel('Sample')
            plt.xlim([20,880])
            plt.suptitle(title)
        return()

In [3]:
### CIC database
X, sleep_stages, sleep_apnea, sleep_apnea_locs, groups = load_data_cic_new(base_dir='./', file_name='cic_database_train_apnea.pkl')
#n_class = 4
#X_lstm, y_lstm = convert_to_LSTM(X, y, groups, n_class)
#y_cnn = convert_to_CNN(y, n_class)
sleep_stages_cnn = np.array(to_categorical(sleep_stages))
sleep_apnea_cnn = np.array(to_categorical(sleep_apnea))
X = np.swapaxes(X, 1, 2)
del_idx = []
for i in range(X.shape[0]):
        if np.isnan(np.sum(X[i,:,2])):
                del_idx.append(i)
        
X_clean = np.delete(X, del_idx, 0)
sleep_stages_cnn_clean = np.delete(sleep_stages_cnn, del_idx, 0)
sleep_apnea_cnn_clean = np.delete(sleep_apnea_cnn, del_idx, 0)
sleep_apnea_locs_clean = np.delete(sleep_apnea_locs, del_idx, 0)
groups_clean = np.delete(groups, del_idx, 0)

In [ ]:
### CIC DATABASE
%reload_ext tensorboard
from datetime import datetime

def lr_schedule(epoch, lr):
    if epoch > 60 and \
            (epoch - 1) % 10 == 0:
        lr *= 0.1
    return lr

opti_loss = "categorical_crossentropy" # "categorical_crossentropy", weighted_categorical_crossentropy

lr_scheduler = LearningRateScheduler(lr_schedule)
X_train, X_test, y_train, y_test = train_test_split(X_clean, sleep_apnea_cnn_clean, train_size=0.01, test_size=0.01)
print('n_train = ', X_train.shape[0],', n_test = ', X_test.shape[0])

CNN_n_layers=3
CNN_kernel_size=[5,5,5]
CNN_n_nodes=[32,64,128]
pooling_size=[3,3,2]
cnn_strides=[2,2,1]
dil_CNN_n_layers=1
dil_CNN_kernel_size=[3]
dil_CNN_n_nodes=[32]
dil_pooling_size=[3]
dil_cnn_strides=[1]

save_title = str(CNN_n_layers) + str(CNN_kernel_size) + str(cnn_strides) + str(pooling_size) + str(dil_CNN_n_layers) + str(dil_CNN_kernel_size) + str(dil_cnn_strides) + str(dil_pooling_size)
title = str(CNN_n_layers) + ' convolutional layers, kernel size ' + str(CNN_kernel_size) + ', strides ' + str(cnn_strides) + ', pooling size ' + str(pooling_size)
title = title + '\n' + str(dil_CNN_n_layers) + ' dilated convolutional layers, kernel size ' + str(dil_CNN_kernel_size) + ', strides ' + str(dil_cnn_strides) + ', pooling size ' + str(dil_pooling_size)
model = create_CNN((X_train.shape[1], X_train.shape[2], 1), normalize=False, n_class=2, 
                    CNN_n_layers=3, CNN_kernel_size=CNN_kernel_size, CNN_n_nodes=CNN_n_nodes, pooling_size=pooling_size, cnn_strides=cnn_strides,
                    dil_CNN_n_layers=dil_CNN_n_layers, dil_CNN_kernel_size=dil_CNN_kernel_size, dil_CNN_n_nodes=dil_CNN_n_nodes, 
                    dil_pooling_size=dil_pooling_size, dil_cnn_strides=dil_cnn_strides, 
                    CNN_n_feature=64, drop_rate=0.2, weight=1e-3)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=opti_loss, metrics=['categorical_accuracy'], weighted_metrics=['categorical_accuracy'])
model.summary()

history = model.fit(X_train, y_train, epochs=100, callbacks=[lr_scheduler], verbose=2,  validation_data=(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)
accu, sensi, speci = accu_sensi_speci(y_pred, y_test)
print(accu, sensi, speci)
save_title = str(CNN_n_layers) + str(CNN_kernel_size) + str(cnn_strides) + str(pooling_size) + str(dil_CNN_n_layers) + str(dil_CNN_kernel_size) + str(dil_cnn_strides) + str(dil_pooling_size)
title = str(CNN_n_layers) + ' convolutional layers, kernel size ' + str(CNN_kernel_size) + ', strides ' + str(cnn_strides) + ', pooling size ' + str(pooling_size)
title = title + '\n' + str(dil_CNN_n_layers) + ' dilated convolutional layers, kernel size ' + str(dil_CNN_kernel_size) + ', strides ' + str(dil_cnn_strides) + ', pooling size ' + str(dil_pooling_size)

plt.figure(figsize=[10, 6])
plt.plot(history.epoch, history.history['categorical_accuracy'], label='categorical_accuracy', color='blue')
#plt.plot(history.epoch, history.history['weighted_categorical_accuracy'], label='weighted_categorical_accuracy', color='blue', linestyle='--')
plt.plot(history.epoch, history.history['val_categorical_accuracy'], label='val_categorical_accuracy', color='orange')
#plt.plot(history.epoch, history.history['val_weighted_categorical_accuracy'], label='val_weighted_categorical_accuracy', color='orange', linestyle='--')
plt.title(title)
plt.axvline(70)
plt.axvline(80)
plt.axvline(90)
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.ylim([0.5,1])
plt.xlim([0,100])
plt.savefig(save_title + '.jpg')
plt.show()


conf_matrix_CNN(X_test, y_test, model) 
title = 'accuracy ' + str(np.round(accu, 2)) + ', sensibility ' + str(np.round(sensi, 2)) + ', specificity ' + str(np.round(speci, 2))
plt.title(title)
plt.savefig(save_title + 'mat.jpg')
plt.show()